<a href="https://colab.research.google.com/github/mikimizutani/food-recognition/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
%cd /content/drive/My Drive/food-recognition/

Mounted at /content/drive/
[Errno 2] No such file or directory: '/content/drive/My Drive/food-recognition/'
/content


In [ ]:
import sys
sys.path.append('/content/drive/My Drive/food-recognition/')
import classes

import os
import torch
from PIL import Image
from pycocotools.coco import COCO
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from classes import FoodDataset
from classes import FocalTverskyLoss
from torch import nn
from torch.utils.data import DataLoader
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
import torchvision
#from engine import train_one_epoch, evaluate
#import utils

In [ ]:
training_path = '/content/drive/My Drive/food-recognition/dataset/train/images.zip'

!ls
!cp "{training_path}" .
!unzip -n images.zip -d "/content/drive/My Drive/food-recognition/dataset/train/"


Streaming output truncated to the last 5000 lines.
  inflating: /content/drive/My Drive/food-recognition/dataset/train/__MACOSX/images/._096441.jpg  
  inflating: /content/drive/My Drive/food-recognition/dataset/train/images/096442.jpg  
  inflating: /content/drive/My Drive/food-recognition/dataset/train/__MACOSX/images/._096442.jpg  
  inflating: /content/drive/My Drive/food-recognition/dataset/train/images/096443.jpg  
  inflating: /content/drive/My Drive/food-recognition/dataset/train/__MACOSX/images/._096443.jpg  
  inflating: /content/drive/My Drive/food-recognition/dataset/train/images/096444.jpg  
  inflating: /content/drive/My Drive/food-recognition/dataset/train/__MACOSX/images/._096444.jpg  
  inflating: /content/drive/My Drive/food-recognition/dataset/train/images/096445.jpg  
  inflating: /content/drive/My Drive/food-recognition/dataset/train/__MACOSX/images/._096445.jpg  
  inflating: /content/drive/My Drive/food-recognition/dataset/train/images/096449.jpg  
  inflating: /

In [ ]:
img = plt.imread("./dataset/train/images/086015.jpg")
plt.imshow(img)

NameError: ignored

In [ ]:
def to_tensor():
    tensors = [torchvision.transforms.Resize((200,200)), torchvision.transforms.ToTensor()]
    return torchvision.transforms.Compose(tensors)


#Method to view image with the categories
def visualise_annotations(coco):
    # nms = set([cat['supercategory'] for cat in cats])
    catIds = coco.getCatIds()
    imgIds = coco.getImgIds()
    img = coco.loadImgs(imgIds[np.random.randint(0, len(imgIds))])[0]
    image = mpimg.imread(os.path.join('./dataset/train/images/', str(img['file_name'])))
    plt.axis('off')
    plt.imshow(image)
    annIds = coco.getAnnIds(imgIds=img['id'], catIds=catIds, iscrowd=None)
    anns = coco.loadAnns(annIds)
    coco.showAnns(anns)
    plt.show()


# def train(model, device, train_loader, optimizer):
#     model.train()
#     y_true = []
#     y_pred = []
#     for i in train_loader:
#
#         # LOADING THE DATA IN A BATCH
#         data, target = i
#
#         # MOVING THE TENSORS TO THE CONFIGURED DEVICE
#         data, target = data.to(device), target.to(device)
#
#         # FORWARD PASS
#         output = model(data.float())
#         loss = FocalTverskyLoss(output)
#         #loss = criterion(output, target.unsqueeze(1))
#
#         # BACKWARD AND OPTIMIZE
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
#
#         # PREDICTIONS
#         pred = np.round(output.detach())
#         target = np.round(target.detach())
#         y_pred.extend(pred.tolist())
#         y_true.extend(target.tolist())
#
#         print("Accuracy on training set is",
#               accuracy_score(y_true, y_pred))


def val(model, device, test_loader):
    # model in eval mode skips Dropout etc
    model.eval()
    y_true = []
    y_pred = []

    # set the requires_grad flag to false as we are in the test mode
    with torch.no_grad():
        for i in test_loader:
            # LOAD THE DATA IN A BATCH
            data, target = i

            # moving the tensors to the configured device
            data, target = data.to(device), target.to(device)

            # the model on the data
            output = model(data.float())

            # PREDICTIONS
            pred = np.round(output)
            target = target.float()
            y_true.extend(target.tolist())
            y_pred.extend(pred.reshape(-1).tolist())

    print("Accuracy on test set is", accuracy_score(y_true, y_pred))
    print("***********************************************************")


def accuracy_score(true, pred):
    #TODO
    return


def get_model_instance_segmentation(num_classes):
    # load an instance segmentation model pre-trained pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)
    return model


if __name__ == '__main__':

    training_json = './dataset/train/annotations.json'
    training_path = './dataset/train/images/'
    #test_path = './dataset/val/images'
    #test_json = './dataset/val/annotations.json'

    

    # dataDir = './dataset'
    # dataType = 'train'
    # annFile = './dataset/train/annotations.json'.format(dataDir, dataType)
    # coco = COCO(annFile)
    #
    # cats = coco.loadCats(coco.getCatIds())
    # nms = [cat['name'] for cat in cats]
    # print('COCO categories: \n{}\n'.format(' '.join(nms)))
    # print(len(nms))

    #visualise_annotations()
    num_epochs = 10

    training_data = FoodDataset(root=training_path, annotation=training_json, transforms=to_tensor())
    #validation_data = FoodDataset(root=test_path, annotation=test_json, transforms=to_tensor())
    training_dataloader = DataLoader(training_data, batch_size=8, shuffle=True, collate_fn=training_data.collate_fn)
    #validation_dataloader = DataLoader(validation_data, batch_size=64, shuffle=True)

    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    num_classes = 6
    # get the model using our helper function
    model = get_model_instance_segmentation(num_classes)

    # move model to the right device
    model.to(device)

    # construct an optimizer
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=0.005,
                                momentum=0.9, weight_decay=0.0005)
    # and a learning rate scheduler
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                   step_size=3,
                                                   gamma=0.1)

    for epoch in range(num_epochs):
        model.train()
        i = 0
        for imgs, annotations in training_dataloader:
            i += 1
            imgs = list(img.to(device) for img in imgs)
            annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
            loss_dict = model(imgs, annotations)
            losses = sum(loss for loss in loss_dict.values())

            optimizer.zero_grad()
            losses.backward()
            optimizer.step()

            print(f'Iteration: {i}/{len(training_dataloader)}, Loss: {losses}')

    print("That's it!")

loading annotations into memory...
Done (t=1.78s)
creating index...
index created!
{'boxes': tensor([[163.6475,  52.5137, 875.6362, 875.6362],
        [ 32.5000,  32.5000, 827.5000, 930.5000]]), 'labels': tensor([1, 1]), 'image_id': tensor([104254]), 'area': tensor([267480., 355631.]), 'iscrowd': tensor([0, 0])}
{'boxes': tensor([[189.8038, 189.8038, 453.8525, 379.0088],
        [226.3275, 226.9263, 329.9113, 329.9113],
        [146.6938, 128.7312, 477.8025, 455.6487],
        [167.0513, 167.0513, 474.8087, 474.8087],
        [ 58.0788,  58.0788, 385.5950, 279.6162],
        [111.9662, 111.9662, 344.8800, 252.0737],
        [  0.5987,   0.5987, 251.4750,  83.2262]]), 'labels': tensor([1, 1, 1, 1, 1, 1, 1]), 'image_id': tensor([39761]), 'area': tensor([ 3611.,  2736., 15374., 27868.,  8461., 13370.,  8925.]), 'iscrowd': tensor([0, 0, 0, 0, 0, 0, 0])}
{'boxes': tensor([[ 93.5000,  93.5000, 298.5000, 304.5000]]), 'labels': tensor([1]), 'image_id': tensor([76855]), 'area': tensor([17805.])

AssertionError: ignored